### True Learning Objectives

- How can I process data in Python

#### How do I process data elements in a DataFrame

With the knowledge of **loop** (repetitive action) and **if..else** (conditional statements), we return our focus on DataFrame, this time to examine how to perform data cleaning and preprocessing tasks. 

We will be using the breast cancer diagnostic data set from https://archive.ics.uci.edu/ml/datasets/breast+cancer+wisconsin+(original)

Let's start by looking at our data. 

In [ ]:
import pandas as pd
data = pd.read_csv('data/breast-cancer-wisconsin.csv', header=None)
data.columns = ['Sample code', 'Clump Thickness', 'Uniformity of Cell Size', 'Uniformity of Cell Shape',
                'Marginal Adhesion', 'Single Epithelial Cell Size', 'Bare Nuclei', 'Bland Chromatin',
                'Normal Nucleoli', 'Mitoses','Class']

print('Number of instances = %d' % (data.shape[0]))
print('Number of attributes = %d' % (data.shape[1]))
data.head()

To address some issues such as duplication, we will go ahead and drop the `Sample code` column, which can be used to identify unique records in this data set. 

In [ ]:
data = data.drop(['Sample code'],axis=1)
data.head()

### Missing Values

From the data's website (https://archive.ics.uci.edu/ml/datasets/breast+cancer+wisconsin+(original)), it shows that there are missing values in the data set. Looking at the data's description (https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.names), the missing values are replaced by **?**. 

This does not impact how Pandas sees the majority of data attributes for each records:

In [ ]:
data.info()

However, it impacts how calculations are performed on each column

In [ ]:
import numpy as np
print(np.sum(np.asarray([0,1,2,3,4], dtype='int64')))
print(np.sum(data['Bare Nuclei']))

We want to replace **?**, which is a character, with a notation from `Numpy` to indicate that this is a missing numerical value. This notation in `Numpy` is `NaN` (not a number). 

In [ ]:
data = data.replace('?',np.NaN)

print('Number of instances = %d' % (data.shape[0]))
print('Number of attributes = %d' % (data.shape[1]))

print('Number of missing values:')
for col in data.columns:
    print('\t%s: %d' % (col,data[col].isna().sum()))
print (data.info())

Only column `Bare Nuclei` has missing values, and after modification, it changes from a numerical column to a generic object column. 

In [ ]:
print(np.sum(data['Bare Nuclei']))

DataFrame allows you to convert the data type

In [ ]:
data['Bare Nuclei'] = pd.to_numeric(data['Bare Nuclei'])
print(np.sum(data['Bare Nuclei']))
print('Number of missing values:')
for col in data.columns:
    print('\t%s: %d' % (col,data[col].isna().sum()))
print (data.info())

Now that we know where the missing values are, how can we preprocess them?
- Replace missing values with median values
- Omit missing values

In [ ]:
data2 = data['Bare Nuclei']

print('Before replacing missing values:')
print(data2[20:25])
data2 = data2.fillna(data2.median())

print('\nAfter replacing missing values:')
print(data2[20:25])

In [ ]:
data3 = data['Bare Nuclei']
print('Before replacing missing values:')
print(data3[20:25])

data3 = data3.dropna()

print('\nAfter replacing missing values:')
print(data3[20:25])

Let's drop the outliers

In [ ]:
data = data.dropna()

### Outliers

Outliers are data instances with characteristics that are considerably different from the rest of the dataset. Outliers can be observed visually using boxplot. 

In [ ]:
%matplotlib inline

#data2 = data.drop(['Class'],axis=1)
data.boxplot(figsize=(20,3))

To discard the outliers, we can compute the Z-score for each attribute and remove those instances containing attributes with abnormally high or low Z-score (e.g., if Z > 3 or Z <= -3).

In [ ]:
z = (data-data.mean())/data.std()
print (z[20:25])

In [ ]:
from scipy import stats
z1 = stats.zscore(data)
print (z1[20:25])

In [ ]:
print(z.shape)
print(data.shape)

**SciPy** to the rescue

In [ ]:
from scipy import stats
import numpy as np

data_no_outlier = data[(np.abs(stats.zscore(data)) < 3).all(axis=1)]
print(data_no_outlier.shape)

In [ ]:
dups = data.duplicated()
print('Number of duplicate rows = %d' % (dups.sum()))
data.loc[[11,28]]

We can drop duplicates:

In [ ]:
print('Number of rows before discarding duplicates = %d' % (data.shape[0]))
data = data.drop_duplicates()
print('Number of rows after discarding duplicates = %d' % (data.shape[0]))

**Aggregation (slide 3)**

Data aggregation is a preprocessing task where the values of two or more objects are combined into a single object. The motivation for aggregation includes (1) reducing the size of data to be processed, (2) changing the granularity of analysis (from fine-scale to coarser-scale), and (3) improving the stability of the data.

In the example below, we will use the daily precipitation time series data for a weather station located at Detroit Metro Airport. The raw data was obtained from the Climate Data Online website (https://www.ncdc.noaa.gov/cdo-web/). The daily precipitation time series will be compared against its monthly values.

In [ ]:
daily = pd.read_csv('data/DTW_prec.csv', header='infer')
print(daily.info())
print(daily.head())

We convert this DataFrame into a `Series`, which is an indexed vector structure in Pandas. 

In [ ]:
daily.index = pd.to_datetime(daily['DATE'])
daily = daily['PRCP']
print(daily.head())

In [ ]:
ax = daily.plot(kind='line',figsize=(15,3))
ax.set_title('Daily Precipitation (variance = %.4f)' % (daily.var()))

Pandas allows us to quickly changing the granularity of analysis through function `groupby`. The documentation of groupby for `Series` is at:
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.groupby.html

In [ ]:
monthly = daily.groupby(pd.Grouper(freq='M')).sum()
ax = monthly.plot(kind='line',figsize=(15,3))
ax.set_title('Monthly Precipitation (variance = %.4f)' % (monthly.var()))

## Question 5:
Modify the code below so that it groups the time series data by year. 
*Hint: If month is M, what will year be?*

In [ ]:
monthly = daily.groupby(pd.Grouper(freq='__')).sum()
ax = monthly.plot(kind='line',figsize=(15,3))
ax.set_title('Monthly Precipitation (variance = %.4f)' % (monthly.var()))

**Sampling (slide 7)**

Sampling is an approach commonly used to facilitate 
- data reduction for exploratory data analysis and scaling up algorithms to big data applications and
- quantifying uncertainties due to varying data distributions. 

There are various methods available for data sampling, such as sampling without replacement, where each selected instance is removed from the dataset, and sampling with replacement, where each selected instance is not removed, thus allowing it to be selected more than once in the sample.

In the example below, we will apply sampling with replacement and without replacement to the breast cancer dataset obtained from the UCI machine learning repository.

#### Without replacement (no possible duplication)

In [ ]:
tmp_data = data.head(10)
sample = tmp_data.sample(n=4)
sample

We can also specify the fraction of data we want to sample

In [ ]:
sample = tmp_data.sample(frac=0.25)
sample

#### With replacement (there could be duplications)

In [ ]:
sample = tmp_data.sample(frac=0.3, replace = True)
sample